## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [2]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

In [3]:
# template 정의
template = "{country}의 수도는 어디인가요?"

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [7]:
prompt = PromptTemplate.from_template("{topic} 에 대해서 쉽게 설명해주세요.")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
chain = prompt | model

## invoke() 호출
- python 딕셔너리 형태로 입력값 전달.(키:값)
- invoke() 함수 호출 시 입력값을 전달

In [8]:
input = {"topic": "인공지능 모델의 학습 원리"}

In [9]:
chain.invoke(input)

AIMessage(content="인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진이 포함된 데이터셋이 필요합니다.\n\n2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 가공해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다.\n\n3. **모델 선택**: 학습할 모델의 구조를 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 다양한 형태가 있을 수 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.\n\n4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터(예: 고양이 사진)와 정답(예: '고양이')을 비교하여, 얼마나 잘 맞추는지를 평가합니다. 이 평가 결과를 바탕으로 모델의 내부 파라미터를 조정하여 점점 더 정확하게 예측할 수 있도록 합니다. 이 과정을 반복합니다.\n\n5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(훈련에 사용하지 않은 데이터)를 사용합니다. 이를 통해 모델이 실제 상황에서도 잘 작동하는지를 확인합니다.\n\n6. **배포 및 활용**: 모델이 충분히 학습하고 검증되면, 실제 환경에서 사용할 수 있도록 배포합니다. 이후에도 새로운 데이터로 모델을 업데이트하거나 개선할 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다. 쉽게 말해, 많은 예시를 보고 배우면서 점점 더 똑똑해지는 과정이라고 할 수 있습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage':

In [11]:
answer = chain.stream(input)
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.

2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 변환해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 과정이 필요합니다.

3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 다양한 종류가 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.

4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨) 간의 관계를 찾아내려고 합니다. 예를 들어, 고양이와 개의 이미지를 분류하는 모델은 각각의 특징을 학습하게 됩니다.

5. **손실 함수**: 모델의 예측이 실제 정답과 얼마나 차이가 있는지를 측정하는 손실 함수를 사용합니다. 이 값을 최소화하는 방향으로 모델의 파라미터(가중치)를 조정합니다.

6. **최적화**: 경량화된 알고리즘(예: 경사 하강법)을 사용하여 손실 함수를 최소화하는 방향으로 모델의 파라미터를 업데이트합니다. 이 과정을 여러 번 반복하면서 모델이 점점 더 정확해집니다.

7. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않은 데이터여야 합니다.

8. **배포**: 모델이 충분히 잘 학습되었다면, 실제 환경에서 사용할 수 있도록 배포합니다. 이후에도 새로운 데이터로 모델을 업데이트하거나 개선할 수 있습니다.

이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다.

## 출력파서(Output Parser)

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [13]:
chain = prompt | model | output_parser

In [14]:
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 종종 불완전하거나 노이즈가 포함되어 있습니다. 따라서 데이터를 정리하고, 필요한 형식으로 변환하는 과정을 거칩니다. 예를 들어, 이미지의 크기를 맞추거나, 텍스트를 정제하는 작업이 포함됩니다.\n\n3. **모델 선택**: 학습할 모델을 선택합니다. 이는 특정 문제를 해결하기 위해 설계된 알고리즘입니다. 예를 들어, 이미지 인식을 위한 CNN(합성곱 신경망)이나 자연어 처리를 위한 RNN(순환 신경망) 등이 있습니다.\n\n4. **학습**: 모델은 데이터를 통해 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 그에 대한 정답(레이블)을 비교하여 오차를 계산합니다. 이 오차를 줄이기 위해 모델의 내부 파라미터(가중치)를 조정합니다. 이 과정을 여러 번 반복하면서 모델은 점점 더 정확한 예측을 할 수 있게 됩니다.\n\n5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않았던 데이터로, 모델이 실제로 얼마나 잘 작동하는지를 확인하는 데 사용됩니다.\n\n6. **배포 및 활용**: 모델이 충분히 잘 학습되었다면, 실제 환경에 배포하여 사용합니다. 예를 들어, 이미지 인식 앱이나 챗봇 등으로 활용될 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다. 요약하자면, 인공지능 모델은 데이터를 통해 학습하고, 그 학습한 내용을 바탕으로 새로운 상황에 대해 예측하거나 결정을 내리는 시스템입니다.'

In [15]:
answer = chain.stream(input)
stream_response(answer)

인공지능 모델의 학습 원리는 크게 두 가지 단계로 나눌 수 있습니다: **훈련(Training)**과 **추론(Inference)**입니다. 이를 쉽게 설명해볼게요.

### 1. 훈련(Training)
훈련 단계에서는 모델이 데이터를 통해 학습합니다. 이 과정은 다음과 같은 단계로 이루어집니다.

- **데이터 수집**: 모델이 학습할 데이터를 모읍니다. 예를 들어, 고양이와 개의 사진을 모아서 분류하는 모델을 만든다고 가정해봅시다.

- **데이터 전처리**: 수집한 데이터를 모델이 이해할 수 있는 형태로 변환합니다. 예를 들어, 이미지를 크기를 조정하거나 색상을 변환하는 작업이 포함됩니다.

- **모델 선택**: 어떤 종류의 인공지능 모델을 사용할지 결정합니다. 예를 들어, 신경망(Neural Network) 같은 모델을 선택할 수 있습니다.

- **학습**: 모델이 데이터를 통해 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터(예: 고양이 사진)와 정답(예: '고양이'라는 레이블)을 비교하여 오차를 계산하고, 이 오차를 줄이기 위해 모델의 내부 파라미터를 조정합니다. 이 과정을 여러 번 반복하면서 모델은 점점 더 정확하게 예측할 수 있게 됩니다.

### 2. 추론(Inference)
훈련이 끝난 후, 모델은 새로운 데이터를 입력받아 예측을 합니다. 이 단계는 다음과 같습니다.

- **새로운 데이터 입력**: 모델이 훈련 중에 보지 못했던 새로운 데이터를 입력합니다.

- **예측**: 모델은 입력된 데이터를 바탕으로 예측을 수행합니다. 예를 들어, 새로운 사진이 고양이인지 개인지 판단합니다.

- **결과 출력**: 모델의 예측 결과를 사용자에게 제공합니다.

### 요약
인공지능 모델은 데이터를 통해 학습하고, 학습한 내용을 바탕으로 새로운 데이터를 예측하는 방식으로 작동합니다. 이 과정에서 많은 데이터와 반복적인 학습이 중요하며, 이를 통해 모델의 성능이 향상됩니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [16]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

# 상황:
{question}

#FORMAT:
- 영어 회화 : 
- 한글 해석 : 
"""

prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(model_name="gpt-4o-mini")

output_parser = StrOutputParser()

In [17]:
chain = prompt | model | output_parser

In [18]:
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화: 
  - Customer: Hello! Could I see the menu, please?
  - Waiter: Of course! Here you go. Can I recommend our special today?
  - Customer: Yes, please! What is it?
  - Waiter: It's grilled salmon with vegetables. It's very popular.
  - Customer: That sounds delicious! I’ll have that, please.
  - Waiter: Great choice! Would you like anything to drink?
  - Customer: Yes, I’d like a glass of red wine, please.
  - Waiter: Perfect! I’ll put that order in for you.

- 한글 해석: 
  - 고객: 안녕하세요! 메뉴를 보여주실 수 있나요?
  - 웨이터: 물론입니다! 여기 있습니다. 오늘의 특별 메뉴를 추천해 드릴까요?
  - 고객: 네, 부탁드립니다! 뭔가요?
  - 웨이터: 구운 연어와 채소입니다. 매우 인기가 많습니다.
  - 고객: 맛있겠네요! 그걸로 주세요.
  - 웨이터: 좋은 선택입니다! 음료는 무엇을 드릴까요?
  - 고객: 네, 레드 와인 한 잔 주세요.
  - 웨이터: 완벽합니다! 주문해 드리겠습니다.


In [19]:
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})

stream_response(answer)

- 영어 회화:  
**Customer:** Hi, I’d like to see the menu, please.  
**Waiter:** Of course! Here you go. Do you have any questions about the menu?  
**Customer:** Yes, what do you recommend?  
**Waiter:** Our spaghetti carbonara is very popular, and the grilled salmon is also a great choice.  
**Customer:** I’ll have the spaghetti carbonara, please. And can I get a side salad with that?  
**Waiter:** Sure! Would you like any drinks with your meal?  
**Customer:** Yes, I’ll have a glass of red wine, please.  
**Waiter:** Great choice! I’ll be right back with your order.

- 한글 해석:  
**고객:** 안녕하세요, 메뉴를 보여주세요.  
**웨이터:** 물론이죠! 여기 있습니다. 메뉴에 대해 질문이 있으신가요?  
**고객:** 네, 어떤 것을 추천하시나요?  
**웨이터:** 우리가 제공하는 스파게티 카르보나라가 매우 인기가 좋고, 그릴에 구운 연어도 훌륭한 선택입니다.  
**고객:** 스파게티 카르보나라로 주세요. 그리고 사이드 샐러드 하나 추가해도 될까요?  
**웨이터:** 물론입니다! 식사와 함께 어떤 음료를 드릴까요?  
**고객:** 네, 레드 와인 한 잔 주세요.  
**웨이터:** 훌륭한 선택입니다! 주문하신 대로 바로 가져다 드리겠습니다.

In [20]:
answer = chain.stream({"question":"미국에서 피자 주문"})
stream_response(answer)

- 영어 회화 : 
Customer: Hi, I’d like to place an order for a pizza, please.  
Pizza Shop: Sure! What size do you want?  
Customer: I’ll take a large, please.  
Pizza Shop: Great! What toppings would you like?  
Customer: I’d like pepperoni and mushrooms.  
Pizza Shop: Would you like any extra cheese?  
Customer: Yes, please! How long will it take to prepare?  
Pizza Shop: It should be ready in about 20 minutes.  
Customer: Perfect! Can you deliver it to 123 Maple Street?  
Pizza Shop: Absolutely! What’s your name?  
Customer: It’s Alex.  
Pizza Shop: Thanks, Alex! Your order will be on its way soon.

- 한글 해석 :  
고객: 안녕하세요, 피자를 주문하고 싶습니다.  
피자 가게: 물론이죠! 어떤 사이즈로 할까요?  
고객: 큰 것 하나 주세요.  
피자 가게: 좋습니다! 어떤 토핑을 원하시나요?  
고객: 페퍼로니와 버섯이요.  
피자 가게: 추가 치즈는 필요하신가요?  
고객: 네, 주세요! 준비하는 데 얼마나 걸리나요?  
피자 가게: 약 20분 정도 걸릴 거예요.  
고객: 완벽해요! 123 메이플 스트리트로 배달해 주실 수 있나요?  
피자 가게: 물론입니다! 성함이 어떻게 되시나요?  
고객: 알렉스입니다.  
피자 가게: 감사합니다, 알렉스! 주문이 곧 출발할 거예요.